## AutoEncoder Copy

In [ ]:
import requests
import os
import time
import tempfile
import subprocess
import re
import collections
import pickle
import ggwave  # Add explicit import for ggwave

# Add TextCompressor class definition
class TextCompressor:
    """
    A text compression system that uses frequency analysis of words, syllables, 
    and character patterns to create an optimized encoding dictionary.
    """
    
    def __init__(self, load_path=None):
        """Initialize the compressor with an optional pre-trained dictionary"""
        if load_path and os.path.exists(load_path):
            self.load_dictionary(load_path)
        else:
            # Dictionary mapping patterns to their encoded representations
            self.encoding_dict = {}
            # Reverse mapping for decoding
            self.decoding_dict = {}
            # Reserve first 32 characters (non-printable ASCII) as encoding markers
            self.next_code = 1
            # Special marker to identify compressed content
            self.COMPRESSION_MARKER = chr(0)
    
    def train(self, corpus, min_frequency=5, max_patterns=256):
        """
        Train the compressor on a text corpus to identify common patterns
        
        Args:
            corpus: Text to analyze for patterns
            min_frequency: Minimum frequency for a pattern to be included
            max_patterns: Maximum number of patterns to encode
        """
        # Tokenize the corpus into words and analyze frequency
        words = re.findall(r'\w+', corpus.lower())
        word_freq = collections.Counter(words)
        
        # Find common character sequences (n-grams)
        ngrams = []
        for n in range(2, 5):  # 2-grams, 3-grams, 4-grams
            for i in range(len(corpus) - n + 1):
                ngrams.append(corpus[i:i+n])
        ngram_freq = collections.Counter(ngrams)
        
        # Combine word and ngram frequencies
        combined_freq = {}
        for item, freq in word_freq.items():
            if len(item) > 2 and freq >= min_frequency:  # Only include words longer than 2 chars
                # Calculate space saving: frequency * (len + 1 for space - 1 for code)
                saving = freq * len(item)
                combined_freq[item] = (freq, saving)
                
        for item, freq in ngram_freq.items():
            if freq >= min_frequency and item not in combined_freq:
                saving = freq * (len(item) - 1)
                combined_freq[item] = (freq, saving)
        
        # Sort by space saving potential
        sorted_patterns = sorted(combined_freq.items(), key=lambda x: x[1][1], reverse=True)
        
        # Take top patterns, limited by max_patterns
        top_patterns = sorted_patterns[:max_patterns]
        
        # Create encoding dictionary using control characters (1-31) and extended ASCII
        self.encoding_dict = {}
        self.decoding_dict = {}
        
        for i, (pattern, _) in enumerate(top_patterns):
            # Start at 1 to avoid NULL character (0), use extended ASCII after 31
            code = chr(1 + i) if i < 31 else chr(128 + (i - 31))
            self.encoding_dict[pattern] = code
            self.decoding_dict[code] = pattern
        
        # Always include the compression marker
        self.COMPRESSION_MARKER = chr(0)
        
        return self.encoding_dict
    
    def encode(self, text):
        """
        Compress text using the trained dictionary
        
        Args:
            text: The text to compress
            
        Returns:
            Compressed text with the compression marker prefix
        """
        if not self.encoding_dict:
            return text  # Can't compress without a dictionary
        
        compressed = text
        
        # Sort patterns by length (longest first) to avoid partial matches
        patterns = sorted(self.encoding_dict.keys(), key=len, reverse=True)
        
        # Replace each pattern with its code
        for pattern in patterns:
            code = self.encoding_dict[pattern]
            # For words, ensure we're replacing whole words
            if re.match(r'^\w+$', pattern):
                compressed = re.sub(r'\b' + re.escape(pattern) + r'\b', code, compressed, flags=re.IGNORECASE)
            else:
                compressed = compressed.replace(pattern, code)
        
        # Add compression marker to indicate this is compressed
        return self.COMPRESSION_MARKER + compressed
    
    def decode(self, compressed_text):
        """
        Decompress text using the dictionary
        
        Args:
            compressed_text: Compressed text starting with the compression marker
            
        Returns:
            Decompressed text or original if not compressed
        """
        # Check if this is actually compressed
        if not compressed_text or compressed_text[0] != self.COMPRESSION_MARKER:
            return compressed_text
        
        # Remove the compression marker
        text = compressed_text[1:]
        
        # Replace each code with its pattern
        for code, pattern in self.decoding_dict.items():
            text = text.replace(code, pattern)
        
        return text
    
    def save_dictionary(self, path):
        """Save the encoding dictionary to a file"""
        data = {
            'encoding_dict': self.encoding_dict,
            'decoding_dict': self.decoding_dict,
            'compression_marker': ord(self.COMPRESSION_MARKER)
        }
        with open(path, 'wb') as f:
            pickle.dump(data, f)
    
    def load_dictionary(self, path):
        """Load the encoding dictionary from a file"""
        with open(path, 'rb') as f:
            data = pickle.load(f)
        
        self.encoding_dict = data['encoding_dict']
        self.decoding_dict = data['decoding_dict']
        self.COMPRESSION_MARKER = chr(data['compression_marker'])
    
    def compression_stats(self, original, compressed):
        """Calculate compression statistics"""
        original_size = len(original.encode('utf-8'))
        compressed_size = len(compressed.encode('utf-8'))
        ratio = original_size / compressed_size if compressed_size > 0 else 0
        
        return {
            'original_size': original_size,
            'compressed_size': compressed_size,
            'compression_ratio': ratio,
            'space_saving': (1 - 1/ratio) * 100 if ratio > 0 else 0
        }

def create_deep_corpus(output_file="deep_corpus.txt", corpus_size=100000000000000000):
    """
    Create a deep English corpus by downloading classic literature texts
    from Project Gutenberg. This creates a rich training set for compression.
    
    Args:
        output_file: File to save the corpus to
        corpus_size: Approximate size of corpus in characters
    
    Returns:
        Path to the created corpus file
    """
    
    # List of classic literature texts from Project Gutenberg (public domain)
    gutenberg_texts = [
    # Classic Literature - Fiction
    "https://www.gutenberg.org/files/1342/1342-0.txt",  # Pride and Prejudice by Jane Austen
    "https://www.gutenberg.org/files/84/84-0.txt",      # Frankenstein by Mary Shelley
    "https://www.gutenberg.org/files/98/98-0.txt",      # A Tale of Two Cities by Charles Dickens
    "https://www.gutenberg.org/files/2701/2701-0.txt",  # Moby Dick by Herman Melville
    "https://www.gutenberg.org/files/345/345-0.txt",    # Dracula by Bram Stoker
    "https://www.gutenberg.org/files/76/76-0.txt",      # Adventures of Huckleberry Finn by Mark Twain
    "https://www.gutenberg.org/files/1661/1661-0.txt",  # Sherlock Holmes by Arthur Conan Doyle
    "https://www.gutenberg.org/files/174/174-0.txt",    # The Picture of Dorian Gray by Oscar Wilde
    "https://www.gutenberg.org/files/145/145-0.txt",    # Middlemarch by George Eliot
    
    # Classic Literature - Additional Genres
    "https://www.gutenberg.org/files/2641/2641-0.txt",  # A Room with a View by E.M. Forster
    "https://www.gutenberg.org/files/11/11-0.txt",      # Alice's Adventures in Wonderland by Lewis Carroll
    "https://www.gutenberg.org/files/1400/1400-0.txt",  # Great Expectations by Charles Dickens
    "https://www.gutenberg.org/files/768/768-0.txt",    # Wuthering Heights by Emily Brontë
    "https://www.gutenberg.org/files/2814/2814-0.txt",  # Dubliners by James Joyce
    
    # Science and Philosophy
    "https://www.gutenberg.org/files/1228/1228-0.txt",  # On the Origin of Species by Charles Darwin
    "https://www.gutenberg.org/files/5001/5001-0.txt",  # Critique of Pure Reason by Immanuel Kant
    "https://www.gutenberg.org/files/4583/4583-0.txt",  # The Theory of Relativity by Albert Einstein
    "https://www.gutenberg.org/files/28233/28233-0.txt", # Mathematical Principles of Natural Philosophy by Newton
    "https://www.gutenberg.org/files/33283/33283-0.txt", # Opticks by Isaac Newton
    "https://www.gutenberg.org/files/30142/30142-0.txt", # Discourse on Method by René Descartes
    
    # Historical and Political Texts
    "https://www.gutenberg.org/files/1404/1404-0.txt",  # Wealth of Nations by Adam Smith
    "https://www.gutenberg.org/files/30254/30254-0.txt", # The Communist Manifesto by Marx and Engels
    "https://www.gutenberg.org/files/150/150-0.txt",    # A Vindication of the Rights of Woman by Mary Wollstonecraft
    "https://www.gutenberg.org/files/61/61-0.txt",      # The U.S. Constitution and Bill of Rights
    "https://www.gutenberg.org/files/100/100-0.txt",    # The Complete Works of Abraham Lincoln
    "https://www.gutenberg.org/files/34901/34901-0.txt", # Democracy in America by Alexis de Tocqueville
    
    # Speeches and Rhetoric
    "https://www.gutenberg.org/files/2130/2130-0.txt",  # Washington's Farewell Address
    "https://www.gutenberg.org/files/16/16-0.txt",      # The Declaration of Independence
    "https://www.gutenberg.org/files/35073/35073-0.txt", # Gettysburg Address by Abraham Lincoln
    "https://www.gutenberg.org/files/49593/49593-0.txt", # Winston Churchill's Speeches
    
    # Technical and Scientific
    "https://www.gutenberg.org/files/37134/37134-0.txt", # The Principles of Psychology by William James
    "https://www.gutenberg.org/files/14988/14988-0.txt", # Elements of Chemistry by Antoine Lavoisier
    "https://www.gutenberg.org/files/10662/10662-0.txt", # The Measurement of Intelligence by Lewis Terman
    "https://www.gutenberg.org/files/5000/5000-0.txt",  # The Principles of Mathematics by Bertrand Russell
    "https://www.gutenberg.org/files/5740/5740-0.txt",  # Relativity: The Special and General Theory by Einstein
    
    # Essays and Non-Fiction
    "https://www.gutenberg.org/files/10615/10615-0.txt", # Essays by Ralph Waldo Emerson
    "https://www.gutenberg.org/files/1080/1080-0.txt",  # Walden by Henry David Thoreau
    "https://www.gutenberg.org/files/74/74-0.txt",      # The Adventures of Tom Sawyer by Mark Twain
    "https://www.gutenberg.org/files/4705/4705-0.txt",  # Essays of Francis Bacon
    "https://www.gutenberg.org/files/200/200-0.txt",    # Sense and Sensibility by Jane Austen
    
    # Various Subjects
    "https://www.gutenberg.org/files/2680/2680-0.txt",  # Meditations by Marcus Aurelius
    "https://www.gutenberg.org/files/3300/3300-0.txt",  # An Enquiry Concerning Human Understanding by David Hume
    "https://www.gutenberg.org/files/41/41-0.txt",      # The Strange Case of Dr. Jekyll and Mr. Hyde by R.L. Stevenson
    "https://www.gutenberg.org/files/2600/2600-0.txt",  # War and Peace by Leo Tolstoy
    "https://www.gutenberg.org/files/158/158-0.txt",    # Emma by Jane Austen
]
    
    # Create corpus by downloading and concatenating texts
    corpus_text = ""
    texts_used = 0
    
    for url in gutenberg_texts:
        try:
            print(f"Downloading {url}")
            response = requests.get(url, timeout=10)
            if response.status_code == 200:
                # Extract the text, removing Project Gutenberg header/footer
                text = response.text
                
                # Remove Gutenberg header (everything before the first empty line after "START")
                start_marker = "*** START OF"
                if start_marker in text:
                    start_pos = text.find(start_marker)
                    text = text[start_pos:]
                    # Find the first paragraph break after the marker
                    para_break = text.find("\r\n\r\n")
                    if para_break > 0:
                        text = text[para_break+4:]
                
                # Remove Gutenberg footer (everything after "END")
                end_marker = "*** END OF"
                if end_marker in text:
                    end_pos = text.find(end_marker)
                    text = text[:end_pos]
                
                # Add to corpus
                corpus_text += text + "\n\n"
                texts_used += 1
                
                # Check if we've reached the target size
                if len(corpus_text) >= corpus_size:
                    break
            
        except Exception as e:
            print(f"Error downloading {url}: {e}")
    
    # Save corpus to file
    with open(output_file, 'w', encoding='utf-8') as f:
        f.write(corpus_text)
    
    print(f"Created corpus with {len(corpus_text)} characters from {texts_used} texts")
    print(f"Saved to {output_file}")
    
    return output_file

def train_deep_compressor(corpus_path=None, output_path="deep_compressor.pkl", max_patterns=512):
    """
    Train a TextCompressor on a deep English corpus
    
    Args:
        corpus_path: Path to corpus file (will create one if None)
        output_path: Where to save the trained compressor
        max_patterns: Maximum number of patterns to include in the dictionary
        
    Returns:
        Trained TextCompressor instance
    """
    # Create corpus if needed
    if not corpus_path:
        corpus_path = create_deep_corpus()
    
    print(f"Training compressor on {corpus_path} with max_patterns={max_patterns}")
    
    # Load corpus
    with open(corpus_path, 'r', encoding='utf-8') as f:
        corpus = f.read()
    
    # Create and train compressor
    compressor = TextCompressor()
    
    # Train with a lower minimum frequency for more patterns but higher max_patterns
    start_time = time.time()
    compressor.train(corpus, min_frequency=3, max_patterns=max_patterns)
    training_time = time.time() - start_time
    
    print(f"Training completed in {training_time:.1f} seconds")
    print(f"Trained dictionary has {len(compressor.encoding_dict)} patterns")
    
    # Save the trained compressor
    compressor.save_dictionary(output_path)
    print(f"Saved compressor to {output_path}")
    
    # Show some statistics about the trained patterns
    patterns = list(compressor.encoding_dict.keys())
    word_patterns = [p for p in patterns if re.match(r'^\w+$', p)]
    ngram_patterns = [p for p in patterns if not re.match(r'^\w+$', p)]
    
    print(f"\nDictionary Statistics:")
    print(f"- Words: {len(word_patterns)} ({len(word_patterns)/len(patterns)*100:.1f}%)")
    print(f"- N-grams: {len(ngram_patterns)} ({len(ngram_patterns)/len(patterns)*100:.1f}%)")
    
    # Show some examples
    print("\nTop 10 longest words in dictionary:")
    top_words = sorted(word_patterns, key=len, reverse=True)[:10]
    for word in top_words:
        print(f"  '{word}' -> '{repr(compressor.encoding_dict[word])}'")
    
    print("\nTop 10 longest n-grams in dictionary:")
    top_ngrams = sorted(ngram_patterns, key=len, reverse=True)[:10]
    for ngram in top_ngrams:
        print(f"  '{ngram}' -> '{repr(compressor.encoding_dict[ngram])}'")
    
    return compressor

def test_compression_performance(compressor, text_samples=None):
    """
    Test the compression performance of a trained compressor
    
    Args:
        compressor: Trained TextCompressor instance
        text_samples: List of text samples to test (will use defaults if None)
        
    Returns:
        Average compression ratio
    """
    if text_samples is None:
        text_samples = [
            # Short message
            "This is a short test message.",
            
            # Medium message with common patterns
            "The quick brown fox jumps over the lazy dog. This message should compress well.",
            
            # Long message with literary style
            """In the quiet hours of the morning, when the world is still wrapped in shadows 
            and the first hints of sunlight begin to stretch across the horizon, there is a rare 
            kind of peace that settles over everything. The air feels crisper, cleaner, and filled 
            with a quiet sense of possibility, as though the universe itself is holding its breath 
            before the start of a new day.""",
            
            # Technical message
            """The compression algorithm works by identifying frequent patterns in the text
            and replacing them with shorter codes. This approach is particularly effective
            for natural language text where certain words and phrases appear repeatedly."""
        ]
    
    total_ratio = 0.0
    
    for i, sample in enumerate(text_samples):
        compressed = compressor.encode(sample)
        stats = compressor.compression_stats(sample, compressed)
        
        # Print only the required information
        print(f"Original message: '{sample}'")
        print(f"Compressed message: {repr(compressed)}")
        print(f"Original: {stats['original_size']} bytes, Compressed: {stats['compressed_size']} bytes")
        print(f"Compression ratio: {stats['compression_ratio']:.2f}x")
        print()
        
        total_ratio += stats['compression_ratio']
    
    avg_ratio = total_ratio / len(text_samples)
    return avg_ratio

def send_encoded_message(message, compressor=None):
    """
    Compress and send a message using ggwave
    
    Args:
        message: Text message to compress and send
        compressor: Optional TextCompressor instance (will create or load one if None)
    """
    # Use provided compressor or load a pre-trained one
    if compressor is None:
        if os.path.exists("deep_compressor.pkl"):
            compressor = TextCompressor("deep_compressor.pkl")
        else:
            compressor = train_deep_compressor()
    
    # Compress the message
    original_message = message
    compressed_message = compressor.encode(message)
    
    # Print only the required information
    stats = compressor.compression_stats(original_message, compressed_message)
    print(f"Original message: '{original_message}'")
    print(f"Compressed message: {repr(compressed_message)}")
    print(f"Original: {stats['original_size']} bytes, Compressed: {stats['compressed_size']} bytes")
    print(f"Compression ratio: {stats['compression_ratio']:.2f}x")
    
    try:
        # Initialize ggwave
        instance = ggwave.init()
        
        # Encode the message with ggwave - correct parameter order
        waveform = ggwave.encode(compressed_message, instance)
        
        # Create a temporary file
        with tempfile.NamedTemporaryFile(suffix='.raw', delete=False) as temp_file:
            temp_filename = temp_file.name
            temp_file.write(waveform)
            temp_file.flush()
        
        # Play the audio without additional output
        subprocess.run([
            'play',
            '-t', 'f32',
            '-r', '48000',
            '-c', '1',
            '-b', '32',
            temp_filename
        ], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
        
        # Clean up
        os.unlink(temp_filename)
        
        # Free ggwave
        ggwave.free(instance)
        
    except Exception as e:
        print(f"Error: {e}")
    
    return compressed_message

def listen_for_deep_encoded_messages(duration=60):
    """
    Listen for deeply encoded messages using the trained compressor
    
    Args:
        duration: How long to listen in seconds
    """
    # Load the deep compressor if available
    if os.path.exists("deep_compressor.pkl"):
        compressor = TextCompressor("deep_compressor.pkl")
    else:
        compressor = train_deep_compressor()
    
    # Define a simple listen function
    def listen_for_encoded_messages(duration, compressor):
        """Simple implementation of the listen function"""
        print(f"Listening for encoded messages for {duration} seconds...")
        
        # Initialize ggwave
        instance = ggwave.init()
        
        # Track received messages
        received_messages = set()
        
        try:
            # Create a temporary file for recording
            with tempfile.NamedTemporaryFile(suffix='.raw', delete=False) as temp_file:
                temp_filename = temp_file.name
            
            # Start recording
            sox_process = subprocess.Popen([
                'rec',
                '-t', 'f32',
                '-r', '48000',
                '-c', '1',
                '-b', '32',
                temp_filename
            ], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
            
            # Wait and check for messages
            start_time = time.time()
            end_time = start_time + duration
            last_check_size = 0
            
            while time.time() < end_time:
                time.sleep(1)
                
                try:
                    # Check for new data
                    current_size = os.path.getsize(temp_filename)
                    
                    if current_size > last_check_size:
                        # Read new data
                        with open(temp_filename, 'rb') as f:
                            f.seek(last_check_size)
                            new_data = f.read()
                        
                        last_check_size = current_size
                        
                        # Try to decode
                        result = ggwave.decode(instance, new_data)
                        
                        if result is not None:
                            message = result.decode('utf-8')
                            
                            # Check if compressed
                            is_compressed = message and message[0] == compressor.COMPRESSION_MARKER
                            
                            if is_compressed:
                                original = message
                                message = compressor.decode(message)
                                stats = compressor.compression_stats(message, original)
                                
                                if message not in received_messages:
                                    print(f"Original message: '{message}'")
                                    print(f"Compressed message: {repr(original)}")
                                    print(f"Original: {stats['original_size']} bytes, Compressed: {stats['compressed_size']} bytes")
                                    print(f"Compression ratio: {stats['compression_ratio']:.2f}x")
                                    print()
                                    received_messages.add(message)
                            else:
                                if message not in received_messages:
                                    print(f"Received uncompressed message: {message}")
                                    received_messages.add(message)
                except Exception as e:
                    pass
                    
        finally:
            # Clean up
            if 'sox_process' in locals():
                sox_process.terminate()
            
            if 'temp_filename' in locals() and os.path.exists(temp_filename):
                os.unlink(temp_filename)
            
            ggwave.free(instance)
        
        return list(received_messages)
    
    return listen_for_encoded_messages(duration, compressor)

# Example usage in a notebook
if __name__ == "__main__":
    try:
        # Check if we already have a trained compressor
        if os.path.exists("deep_compressor.pkl"):
            print("Loading existing compressor...")
            compressor = TextCompressor("deep_compressor.pkl")
        else:
            # Create the corpus and train a deep compressor
            print("Creating corpus and training compressor...")
            corpus_path = create_deep_corpus()
            compressor = train_deep_compressor(corpus_path)

        example_message = "In an age where technology evolves at a breakneck pace, the integration of artificial intelligence into daily life has become not just a possibility, but an inevitability that touches nearly every industry and facet of human interaction, transforming the very way people live, work, learn, and relate to one another. From the automation of repetitive, mundane tasks to the development of highly sophisticated systems capable of learning, reasoning, interpreting context, and even producing original content, AI has become a cornerstone of the digital era. In healthcare, machine learning models are being used to predict patient outcomes, identify rare diseases, and streamline administrative tasks, reducing human error and enabling doctors to focus more on personalized care. In education, adaptive learning platforms can assess a student’s strengths and weaknesses in real time, tailoring the curriculum to maximize engagement and understanding while simultaneously providing educators with actionable insights. The financial industry leverages AI to detect fraudulent transactions in milliseconds, model complex market behaviors, and offer personalized investment strategies, while supply chain and logistics companies use predictive algorithms to optimize delivery routes, forecast demand, and reduce costs. Meanwhile, autonomous vehicles, once the stuff of science fiction, are being tested and deployed in real-world environments, navigating complex traffic scenarios with increasing confidence and safety. Even in creative fields like music, writing, and visual arts, AI tools are collaborating with human artists to generate entirely new forms of expression, challenging traditional definitions of creativity and authorship. Smart assistants such as Siri, Alexa, and Google Assistant have become ubiquitous in homes, capable of handling tasks from managing calendars to controlling smart home devices, while large language models like ChatGPT are being utilized in customer support, coding assistance, legal analysis, and mental health applications. Despite these incredible advancements, however, the rise of AI has sparked urgent debates around issues like data privacy, surveillance, algorithmic bias, job displacement, and the potential misuse of autonomous systems. There are growing concerns about the opacity of complex neural networks and the challenge of holding algorithms accountable for decisions that may carry ethical or legal implications. As AI systems are trained on vast datasets, often scraped from the internet, they may inadvertently perpetuate harmful stereotypes or reinforce existing societal inequalities. Furthermore, as automation continues to replace certain job functions, there is a critical need to rethink education, workforce training, and economic policy to ensure that people are equipped to thrive in a world where human and machine collaboration is the norm. Governments, corporations, and communities must work together to establish transparent guidelines and frameworks that prioritize fairness, accountability, and inclusivity in the development and deployment of AI technologies. In doing so, society has the opportunity to shape a future in which artificial intelligence not only enhances productivity and innovation but also serves as a tool for promoting equity, creativity, and human well-being across the globe. The path forward is complex and multifaceted, but with thoughtful design, robust regulation, and a commitment to ethical principles, the vast potential of AI can be harnessed to solve some of humanity’s most pressing challenges while enriching the collective human experience in profound and transformative ways."
        
        # Send the message using the deep compressor
        listen_for_deep_encoded_messages()
    except Exception as e: 
        print(f"Error: {e}")

Loading existing compressor...
Listening for encoded messages for 60 seconds...


Receiving sound data ...
Received end marker. Frames left = 1651, recorded = 155
Analyzing captured data ..
Decoded length = 48, protocol = 'Fast' (1)
Received sound data successfully: ''


Original message: '[1/4] in an age where technology evolves at a breakneck '
Compressed message: '\x00[1/4] \x10\x13\x0bge½ítechnolog\x92evolvesŔa¤reakneck '
Original: 56 bytes, Compressed: 48 bytes
Compression ratio: 1.17x



Receiving sound data ...
Received end marker. Frames left = 1542, recorded = 264
Analyzing captured data ..
Decoded length = 86, protocol = 'Fast' (1)
Received sound data successfully: ''


Original message: '[2/4] pace, the integration of artificial intelligence into daily life has become not just a possibility, '
Compressed message: '\x00[2/4]čacƴ\x01Ɍegration\x19artificialɌelligencȑ\x82daiĆlifƈs®comeî just¦possibilitȆ'
Original: 106 bytes, Compressed: 86 bytes
Compression ratio: 1.23x



Receiving sound data ...
Received end marker. Frames left = 1548, recorded = 258
Analyzing captured data ..
Decoded length = 84, protocol = 'Fast' (1)
Received sound data successfully: ''


Original message: '[3/4] but an inevitability that touches nearly every industry and facet of human interaction, transforming'
Compressed message: '\x00[3/4]ƶ\x13\x95evitabilit\x92\x1f\x1duchūnearĆevery\x95dustȗ\rfacet\x19humanɌeractionƑransforming'
Original: 106 bytes, Compressed: 84 bytes
Compression ratio: 1.26x



Receiving sound data ...
Received end marker. Frames left = 1622, recorded = 184
Analyzing captured data ..
Decoded length = 57, protocol = 'Fast' (1)
Received sound data successfully: ''


Original message: '[4/4]  the very way people live, work, learn, and relate to one another.'
Compressed message: '\x00[4/4] \x01 very\xad\x92peopŦlivìwork\x1alearȊ\rrelate\x14Ǣanother.'
Original: 72 bytes, Compressed: 57 bytes
Compression ratio: 1.26x

